In [37]:
import json
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
import folium
import datetime

from pandas import json_normalize

- channelGrouping : 유입 경로
- date : 방문 날짜
- device : 사용자 장치 정보
    - browser
    - browserVersion
    - browserSize
    - operatingSystem
    - operatingSystemVersion
    - isMobile : T/F
    - mobileDeviceBranding
    - mobileDeviceModel
    - mobileInputSelector
    - mobileDeviceInfo
    - mobileDeviceMarketingName
    - flashVersion
    - language
    - screenColors
    - screenResolution
    - deviceCategory
- fullVisitorId : 사용자 고유 식별자
- geoNetwork : 사용자 위치 정보
    - continent
    - subContinent
    - country
    - region
    - metro
    - city
    - cityId
    - networkDomain
    - latitude
    - longitude
    - networkLocation
- sessionId : fullVisitorId + visitId
- socialEngagementType : 참여 유형
- totals : 세션 전체 집계값(?)
    - visits
    - hits
    - pageviews
    - bounces
    - newVisits
- trafficSource : 유입 트래픽 소스
    - campaign
    - source
    - medium
    - keyword
    - adwordsClickInfo
        - criteriaParameters
    - isTrueDirect
- visitId : 식별자
- visitNumber : 세션 번호
- visitStartTime : 타임스탬프, visitId와 동일

In [38]:
# 'device', 'geoNetwork', 'totals', 'trafficSource’ 컬럼 추출
json_columns = ['device', 'geoNetwork','totals', 'trafficSource']

def load_dataframe(csv_path):
    path = csv_path
    df = pd.read_csv(path, converters={column: json.loads for column in json_columns}, 
                     dtype={'fullVisitorId': 'str'})
   
    for column in json_columns:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

    return df

In [39]:
train = load_dataframe('C:/Users/USER/OneDrive/바탕 화면/내일배움캠프 데이터 분석/train.csv/train.csv')

In [40]:
train.head(3)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,...,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_isTrueDirect,trafficSource_referralPath,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adContent,trafficSource_campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    903653 non-null  object
 1   date                                               903653 non-null  int64 
 2   fullVisitorId                                      903653 non-null  object
 3   sessionId                                          903653 non-null  object
 4   socialEngagementType                               903653 non-null  object
 5   visitId                                            903653 non-null  int64 
 6   visitNumber                                        903653 non-null  int64 
 7   visitStartTime                                     903653 non-null  int64 
 8   device_browser                                     903653 non-null  object
 9   devi

In [42]:
train.describe(include='all')

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,...,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_isTrueDirect,trafficSource_referralPath,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adContent,trafficSource_campaignCode
count,903653,9.036530e+05,903653,903653,903653,9.036530e+05,903653.000000,9.036530e+05,903653,903653,...,903653,274005,330941,21460,21460,21561,21460,21460,10946,1
unique,8,NaN,714167,902755,1,NaN,NaN,NaN,54,1,...,1,1,1475,8,2,17774,2,1,44,1
top,Organic Search,NaN,1957458976293878100,14108533830165900_1482391162,Not Socially Engaged,NaN,NaN,NaN,Chrome,not available in demo dataset,...,not available in demo dataset,True,/,1,Top,Cj0KEQjwmIrJBRCRmJ_x7KDo-9oBEiQAuUPKMufMpuG3Zd...,Google Search,False,Google Merchandise Collection,11251kjhkvahf
freq,381561,NaN,278,2,903653,NaN,NaN,NaN,620364,903653,...,903653,274005,75523,21362,20956,70,21453,21460,5122,1
mean,NaN,2.016589e+07,NaN,NaN,NaN,1.485007e+09,2.264897,1.485007e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,4.697698e+03,NaN,NaN,NaN,9.022124e+06,9.283735,9.022124e+06,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,2.016080e+07,NaN,NaN,NaN,1.470035e+09,1.000000,1.470035e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,2.016103e+07,NaN,NaN,NaN,1.477561e+09,1.000000,1.477561e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,2.017011e+07,NaN,NaN,NaN,1.483949e+09,1.000000,1.483949e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,2.017042e+07,NaN,NaN,NaN,1.492759e+09,1.000000,1.492759e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
train.columns

Index(['channelGrouping', 'date', 'fullVisitorId', 'sessionId',
       'socialEngagementType', 'visitId', 'visitNumber', 'visitStartTime',
       'device_browser', 'device_browserVersion', 'device_browserSize',
       'device_operatingSystem', 'device_operatingSystemVersion',
       'device_isMobile', 'device_mobileDeviceBranding',
       'device_mobileDeviceModel', 'device_mobileInputSelector',
       'device_mobileDeviceInfo', 'device_mobileDeviceMarketingName',
       'device_flashVersion', 'device_language', 'device_screenColors',
       'device_screenResolution', 'device_deviceCategory',
       'geoNetwork_continent', 'geoNetwork_subContinent', 'geoNetwork_country',
       'geoNetwork_region', 'geoNetwork_metro', 'geoNetwork_city',
       'geoNetwork_cityId', 'geoNetwork_networkDomain', 'geoNetwork_latitude',
       'geoNetwork_longitude', 'geoNetwork_networkLocation', 'totals_visits',
       'totals_hits', 'totals_pageviews', 'totals_bounces', 'totals_newVisits',
       'totals_t

In [45]:
trafficSource = train[['trafficSource_campaign',
       'trafficSource_source', 'trafficSource_medium', 'trafficSource_keyword',
       'trafficSource_adwordsClickInfo.criteriaParameters',
       'trafficSource_isTrueDirect', 'trafficSource_referralPath',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.isVideoAd', 'trafficSource_adContent',
       'trafficSource_campaignCode']]
trafficSource.describe(include='all')

,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_isTrueDirect,trafficSource_referralPath,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adContent,trafficSource_campaignCode
count,903653,903653,903653,400724,903653,274005,330941,21460,21460,21561,21460,21460,10946,1
unique,10,380,7,3659,1,1,1475,8,2,17774,2,1,44,1
top,(not set),google,organic,(not provided),not available in demo dataset,True,/,1,Top,Cj0KEQjwmIrJBRCRmJ_x7KDo-9oBEiQAuUPKMufMpuG3Zd...,Google Search,False,Google Merchandise Collection,11251kjhkvahf
freq,865347,400788,381561,366363,903653,274005,75523,21362,20956,70,21453,21460,5122,1


In [50]:
# 사용할 컬럼만 남기기
# trafficSource는 잘 모르겠어요...
train_1 = train[['date', 'visitStartTime','fullVisitorId', 'visitId', 'sessionId', 'channelGrouping', 'visitNumber', 'geoNetwork_continent', 'geoNetwork_country', 'device_browser', 'device_deviceCategory', 'totals_visits', 'totals_hits', 'totals_pageviews', 'totals_bounces', 'totals_newVisits', 'totals_transactionRevenue', 'trafficSource_campaign',
                 'trafficSource_source', 'trafficSource_medium', 'trafficSource_keyword', 'trafficSource_adwordsClickInfo.page', 'trafficSource_adwordsClickInfo.slot', 'trafficSource_adwordsClickInfo.adNetworkType', 'trafficSource_adContent']]

In [53]:
#date 컬럼 date type으로 변경
train_1['date'] = pd.to_datetime(train_1['date'], format='%Y%m%d')

C:\Users\USER\AppData\Local\Temp\ipykernel_29004\2206694138.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['date'] = pd.to_datetime(train_1['date'], format='%Y%m%d')


In [56]:
# POSIX 시간을 date type으로 변경
train_1['visitStartTime'] = pd.to_datetime(train_1['visitStartTime'], unit='s')

C:\Users\USER\AppData\Local\Temp\ipykernel_29004\2538197316.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['visitStartTime'] = pd.to_datetime(train_1['visitStartTime'], unit='s')


In [58]:
train_1.head()

,date,visitStartTime,fullVisitorId,visitId,sessionId,channelGrouping,visitNumber,geoNetwork_continent,geoNetwork_country,device_browser,...,totals_newVisits,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adContent
0,2016-09-02,2016-09-02 15:33:05,1131660440785968503,1472830385,1131660440785968503_1472830385,Organic Search,1,Asia,Turkey,Chrome,...,1,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN
1,2016-09-02,2016-09-03 05:22:27,377306020877927890,1472880147,377306020877927890_1472880147,Organic Search,1,Oceania,Australia,Firefox,...,1,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN
2,2016-09-02,2016-09-03 01:16:26,3895546263509774583,1472865386,3895546263509774583_1472865386,Organic Search,1,Europe,Spain,Chrome,...,1,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN
3,2016-09-02,2016-09-03 05:40:13,4763447161404445595,1472881213,4763447161404445595_1472881213,Organic Search,1,Asia,Indonesia,UC Browser,...,1,NaN,(not set),google,organic,google + online,NaN,NaN,NaN,NaN
4,2016-09-02,2016-09-02 13:23:20,27294437909732085,1472822600,27294437909732085_1472822600,Organic Search,2,Europe,United Kingdom,Chrome,...,NaN,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN


In [59]:
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 25 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   date                                          903653 non-null  datetime64[ns]
 1   visitStartTime                                903653 non-null  datetime64[ns]
 2   fullVisitorId                                 903653 non-null  object        
 3   visitId                                       903653 non-null  int64         
 4   sessionId                                     903653 non-null  object        
 5   channelGrouping                               903653 non-null  object        
 6   visitNumber                                   903653 non-null  int64         
 7   geoNetwork_continent                          903653 non-null  object        
 8   geoNetwork_country                            903653 n

In [60]:
train_1.describe(include='all')

,date,visitStartTime,fullVisitorId,visitId,sessionId,channelGrouping,visitNumber,geoNetwork_continent,geoNetwork_country,device_browser,...,totals_newVisits,totals_transactionRevenue,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adContent
count,903653,903653,903653,9.036530e+05,903653,903653,903653.000000,903653,903653,903653,...,703060,11515,903653,903653,903653,400724,21460,21460,21460,10946
unique,NaN,NaN,714167,NaN,902755,8,NaN,6,222,54,...,1,5332,10,380,7,3659,8,2,2,44
top,NaN,NaN,1957458976293878100,NaN,14108533830165900_1482391162,Organic Search,NaN,Americas,United States,Chrome,...,1,16990000,(not set),google,organic,(not provided),1,Top,Google Search,Google Merchandise Collection
freq,NaN,NaN,278,NaN,2,381561,NaN,450377,364744,620364,...,703060,256,865347,400788,381561,366363,21362,20956,21453,5122
mean,2017-01-20 18:54:58.635206400,2017-01-21 14:01:30.059528448,NaN,1.485007e+09,NaN,NaN,2.264897,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,2016-08-01 00:00:00,2016-08-01 07:00:12,NaN,1.470035e+09,NaN,NaN,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2016-10-27 00:00:00,2016-10-27 09:39:29,NaN,1.477561e+09,NaN,NaN,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,2017-01-09 00:00:00,2017-01-09 08:02:14,NaN,1.483949e+09,NaN,NaN,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,2017-04-21 00:00:00,2017-04-21 07:12:42,NaN,1.492759e+09,NaN,NaN,1.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,2017-08-01 00:00:00,2017-08-02 06:59:53,NaN,1.501657e+09,NaN,NaN,395.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
